In [4]:
import requests 
import pandas as pd
import numpy as np
import math
from pprint import pprint
import re
import os 
from statistics import mean, median, mode
%matplotlib inline
import matplotlib.pyplot as plt
import json

In [5]:
MAX_RECORDS = 200
latmin = 43.6406
latmax = 43.736439
lngmax = -79.252357       
lngmin = -79.549301
records = 2500
records_per_page = min(records, MAX_RECORDS)
pages = math.ceil(records / MAX_RECORDS)

maindf = pd.DataFrame({"Address":''},index=[np.arange(0, records)])
maindf["Sale Price"]= ""    
maindf['Mls Number']=""
maindf['Near By Ammenities']= ""
maindf["Ownership Type"] = ""
maindf["Parking"] = ""
maindf["Bathrooms"] = ""
maindf["Bedrooms"] = ""

for page in np.arange(pages):
    url = 'https://api2.realtor.ca/Listing.svc/PropertySearch_Post'
    body = {
        "CultureId":1,
        "ApplicationId":1,
        "PropertySearchTypeId":1,
        "TransactionTypeID":2,
        "BuildingTypeId":0,
        "LongitudeMin":f'{lngmin}',
        "LongitudeMax":f'{lngmax}',
        "LatitudeMin":f'{latmin}',
        "LatitudeMax":f'{latmax}',
        "AirCondition":1,
        "PriceMin":250000,
        "RecordsPerPage": records_per_page,
        "CurrentPage": page + 1
    }

    response = requests.post(url, data=body).json() 
    
    for index, mls_post in enumerate(response['Results']):
        if page > 0:
            index += (MAX_RECORDS * page)
            
        try:                        
            lng = float(mls_post['Property']['Address']['Longitude'])
            address = mls_post['Property']['Address']
    
            if (lng >= lngmin and lng <= lngmax and address['AddressText'] not in maindf['Address'].unique()):
                maindf.loc[index,"Address"] = address['AddressText']        
                maindf.loc[index,"Mls Number"]= mls_post['MlsNumber']
                maindf.loc[index,"Date Updated"] = mls_post['Individual'][0]['Organization']['PhotoLastupdate']
                    
                try:
                    maindf.loc[index,"Sale Price"] = mls_post['Property']['Price']
                    maindf.loc[index,"Near By Ammenities"] = mls_post['Property']['AmmenitiesNearBy']
                    maindf.loc[index,"Ownership Type"] = mls_post['Property']['OwnershipType']
                    maindf.loc[index,"Parking"] = mls_post['Property']['Parking'][0]['Name']
                    maindf.loc[index,"Bathrooms"] = mls_post['Building']['BathroomTotal']
                    maindf.loc[index,"Bedrooms"] = mls_post['Building']['Bedrooms']
                except:
                    maindf.loc[index,"Sale Price"] = np.nan
                    maindf.loc[index,"Near By Ammenities"] = np.nan
                    maindf.loc[index,"Ownership Type"] = np.nan
                    maindf.loc[index,"Parking"] = np.nan
                    maindf.loc[index,"Bathrooms"] = np.nan
                    maindf.loc[index,"Bedrooms"] = np.nan
                
        except Exception as e:
            print('Error ', e)  
            
maindfclean = maindf.loc[maindf['Address']!="",:]

In [16]:
maindfclean = maindf.dropna(how='any')

In [17]:
# maindfclean = maindf.loc[maindf['Address']!="",:]
maindfclean.set_index('Mls Number')
maindfclean.head()

,Address,Sale Price,Mls Number,Near By Ammenities,Ownership Type,Parking,Bathrooms,Bedrooms,Date Updated
0,"#628 -2737 KEELE ST|Toronto, Ontario M3M2E9","$283,900",W4652611,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,1,2019-12-15 2:24:42 PM
1,"#614 -2737 KEELE ST|Toronto, Ontario M3M2E9","$294,500",W4653141,"Hospital, Public Transit",Condominium/Strata,Underground,1,1,2019-12-13 6:36:20 PM
4,"#910 -2737 KEELE ST|Toronto, Ontario M3M2E9","$339,700",W4641655,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,2,2019-12-05 2:31:41 PM
5,"#907 -234 ALBION RD|Toronto, Ontario M9W6A5","$355,000",W4622564,"Hospital, Park, Schools",Condominium/Strata,Underground,1,2,2019-12-09 8:24:39 AM
7,"#601 -3555 BATHURST ST|Toronto, Ontario M6A2Y8","$365,000",C4637991,"Park, Public Transit, Schools",Condominium/Strata,Underground,2,2,2019-12-06 2:25:51 PM


In [18]:
pattern = re.compile(r'(\d\d\d\d-\d\d-\d\d)')

In [19]:
maindfclean['Date Updated'] = maindfclean['Date Updated'].str.extract(pattern)


C:\Users\HP Book Pro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
maindfclean.head()

,Address,Sale Price,Mls Number,Near By Ammenities,Ownership Type,Parking,Bathrooms,Bedrooms,Date Updated
0,"#628 -2737 KEELE ST|Toronto, Ontario M3M2E9","$283,900",W4652611,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,1,2019-12-15
1,"#614 -2737 KEELE ST|Toronto, Ontario M3M2E9","$294,500",W4653141,"Hospital, Public Transit",Condominium/Strata,Underground,1,1,2019-12-13
4,"#910 -2737 KEELE ST|Toronto, Ontario M3M2E9","$339,700",W4641655,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,2,2019-12-05
5,"#907 -234 ALBION RD|Toronto, Ontario M9W6A5","$355,000",W4622564,"Hospital, Park, Schools",Condominium/Strata,Underground,1,2,2019-12-09
7,"#601 -3555 BATHURST ST|Toronto, Ontario M6A2Y8","$365,000",C4637991,"Park, Public Transit, Schools",Condominium/Strata,Underground,2,2,2019-12-06


In [21]:
maindfclean['Sale Price'] = maindfclean['Sale Price'].str[1:]
maindfclean.head()

C:\Users\HP Book Pro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Address,Sale Price,Mls Number,Near By Ammenities,Ownership Type,Parking,Bathrooms,Bedrooms,Date Updated
0,"#628 -2737 KEELE ST|Toronto, Ontario M3M2E9","283,900",W4652611,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,1,2019-12-15
1,"#614 -2737 KEELE ST|Toronto, Ontario M3M2E9","294,500",W4653141,"Hospital, Public Transit",Condominium/Strata,Underground,1,1,2019-12-13
4,"#910 -2737 KEELE ST|Toronto, Ontario M3M2E9","339,700",W4641655,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,2,2019-12-05
5,"#907 -234 ALBION RD|Toronto, Ontario M9W6A5","355,000",W4622564,"Hospital, Park, Schools",Condominium/Strata,Underground,1,2,2019-12-09
7,"#601 -3555 BATHURST ST|Toronto, Ontario M6A2Y8","365,000",C4637991,"Park, Public Transit, Schools",Condominium/Strata,Underground,2,2,2019-12-06


In [22]:
maindfclean.count()

Address               778
Sale Price            778
Mls Number            778
Near By Ammenities    778
Ownership Type        778
Parking               778
Bathrooms             778
Bedrooms              778
Date Updated          778
dtype: int64